# Chapter 13: Policy Gradient Methods

## Intro:

So far, all methods have been action-value methods. Now, we consider methods that learn a *parameterized policy* that can choose actions without needing to consult a value function. A value function might still be needed to learn the policy parameter, but it isn't required for picking actions. The notation $\theta \in \mathbb{R}^{d'}$ is used for the policy's parameter vector. So we write $\pi(a | s, \theta) = $Pr{$A_t = a | S_t = s, \theta_t = \theta$} to be the probability that action $a$ is taken at time $t$ given that the environment is in state $s$ at time $t$ with parameter $\theta$. If a method also uses a value function, the weight vector of the value function uses the same notation as before. 

The methods considered here learn the policy parameter based on the gradient of some performance measure $J(\theta)$ with respect to the policy parameter. The methods aim to maximize performance, so the updates approximate gradient ascent in J:

$\theta_{t+1} = \theta_t + \alpha \widehat{\nabla J(\theta_t)}$

$\widehat{\nabla J(\theta_t)} \in \mathbb{R}^d$ is a stochastic estimate, its expectation approximates gradient of the performance measure with respect to (w.r.t) argument $\theta_t$. All methods following this general outline are called *policy-gradient methods*, whether or not they also learn an approximate value function. Methods learning both approximations to policy and value functions are called *actor-critic methods*, where *actor* refers to the learned policy, and *critic* is the learned value function. 

## 13.1: Policy Approximation and its Advantages

We can parameterize the policy in any way, as long as $\pi(a | s, \theta)$ is differentiable w.r.t its parameters. That is, as long as $\nabla \pi(a | s, \theta)$ (column vector of partial derivatives of the policy $\pi(a|s, \theta)$ w.r.t the components of $\theta$) exists and is finite for all $s \in S, a \in A(s), \theta \in \mathbb{R}^{d'}$. To deal with exploration vs exploitation, we require that the policy is always stochastic. 

If we've got a discrete action space and a not too large state-space, then a natural way to parameterize is to form parameterized numerical preferences $h(s, a, \theta) \in \mathbb{R}$ for each state-action pair. We want to give the actions with the highest preference in each state the highest probability of being chosen, i.e. according to the exponential softmax distribution:

$\pi(a|s, \theta) \stackrel{.}{=} \frac{e^{h(s, a, \theta)}}{\sum_b e^{h(s, b, \theta)}}$

e is the base of the natural log. The denominator here is exactly what's required to make sure that the probabilities in each state sum up to one. Policy parameterization done in this way is called *softmax in action preferences*. 

The action preferences can be parameterized arbitrarily. A neural network can be used, in this case, $\theta$ might be the vector of all the connection weights in the network, or the preferences could be linear in features:

$h(s, a, \theta) = \theta^T \textbf{x}(s, a)$

using feature vectors $\textbf{x}(s, a) \in \mathbb{R}^{d'}$. An advantage to parameterizing policies with the softmax is that the resulting approximate policy can approach a deterministic policy, compared to with $\varepsilon$-greedy policies when there is always a probability $\epsilon$ of selecting a random action. You could pick actions according to a softmax distribution based on action values, but this wouldn't allow approaching a deterministic policy. The action value estimates would instead converge to their corresponding true values, differing by a finite amount, and so would be real probabilities instead of 0 and 1. 

If softmax distribution is given a temperature parameter, then this parameter can be reduced over time to reach deterministic policy. However, it would be difficult to choose the reduction schedule or the initial temperature value. 

Action preferences differ because they do not approach specific values, instead they are driven to approximate the optimal policy.

If the optimal policy is a deterministic one, then the preference of optimal actions will be infinitely higher than the other actions.

Another bonus of parameterizing policies according to the softmax distribution over action preferences is that it allows us to pick actions with arbitrary probabilities. When dealing with a problem involving significant function approximation, it may be best to approximate a stochastic policy. i.e., in a card game when one has imperfect information, it is often optimal to do two different things with specific probabilities, like when bluffing in poker. Action-value methods don't have a way of naturally finding stochastic policies, but policy approximating methods can. 

A simple advantage that policy approximation might have over action-value methods is that the policy might be a simpler function to approximate. Sometimes the action-value function is simpler to find, but since problems vary in their complexity of policies and action-value functions, other times the policy will be simpler to approximate. When the policy is simpler, a policy based method will normally learn faster and learn an asymptotically superior policy.

### 13.2 The Policy Improvement Theorem

There is an important theoretical advantage to policy paramaterization over $\varepsilon$-greedy action selection; with continuous policy parameterization the action probabilities change smoothly as a function of the learned parameter. This is in contrast to $\varepsilon$-greedy selection where the action probabilities may change dramatically for an arbitrarily small change in the action values, if that results in a different action having the maximal value. 

Mostly due to this, we can make stronger convergence guarantees for policy-gradient methods than for action-value methods. 

Particularly, the continuity of the policy dependence on parameters is what enables policy-gradient methods to approximate gradient ascent. 

The performance measure $J(\theta)$ has to be treated differently for episodic and continuing cases. In the episodic case performance is defined as:

$J(\theta) \stackrel{.}{=} v_{\pi_\theta}(s_0)$

where $v_{\pi_\theta}$ is the true value function for $\pi_\theta$, the policy determined by $\theta$. We will assume no discounting ($\gamma = 1$) for the episodic case. 

Continuous case performance to be discussed in a later section. 

It may appear challenging to change the policy in such a way that it guarantees improvement. The problem is because performance depends both on the actions chosen and the distributions of states in which those actions are made, and that both of these things are affected by the policy parameter. Given a state, the effect of the policy parameter on the actions, and thus on reward, can be computed in a relatively straightforward way from knowledge of the paramaterization. However, the effect of the policy parameter on reward is a function of the environment and is unknown. How to estimate the performance gradient with respect to the policy parameter when the gradient depends on the unknown effect of policy changes to the state distribution?

we use the policy approximation theorem to produce an analytic expression for the gradient of performance w.r.t the policy parameter. (This is what we need to approximate gradient ascent.) The expression does not involve the derivative of the state distribution. Policy gradient theorem for the episodic case establishes:

$\nabla J(\theta) \propto \sum_s \mu(s) \sum_a q_\pi(s, a) \nabla \pi(a | s, \theta) \quad \quad \quad (13.5)$

The gradients are the column vectors of partial derivatives w.r.t the components of $\theta$, $\pi$ is the policy corresponding to parameter vector $\theta$. $\propto$ means "proportional to". When in the episodic case, the constant of proportionality is the average length of an episode, and in the continuous case it is 1, so that the expression is an equality. Distribution $\mu$ is the on-policy distribution under $\pi$. 

### 13.3 REINFORCE: Monte Carlo Policy Gradient

Recall strategy of stochastic gradient ascent from (13.1). It requires a way to get samples where the expectation of the sample gradient is proportional to the actual gradient of the performance measure as a function of the parameter. We only need the sample gradients to be proportional to the gradient because any constant of proportionality can be absorbed into the step size. The policy gradient theorem yields an exact way expression proportional to the gradient. The only thing needed is some way of sampling which has an expectation equal to or approximately equal to this expression. The RHS of the policy gradient is a sum pver states weighted by how often the states occur under policy $\pi$. If $\pi$ is followed, the states will occur in those proportions.

$\nabla J(\theta) \propto \sum_s \mu(s) \sum_a q_\pi(s, a)\nabla \pi(a| s, \theta)$  
$\nabla J(\theta) = \mathbb{E}_\pi [\sum_a q_\pi(S_t, a) \nabla \pi(a| S_t, \theta) \quad \quad \quad (13.6)$ 

At this point we could stop and set up the stochastic gradient ascent as:

$\theta_{t+1} \stackrel{.}{=} \theta_t + \alpha \sum_a \hat{q}(S_t, a, \textbf{w}) \nabla \pi(a|S_t, \theta) \quad \quad \quad (13.7)$

$\hat{q}$ is a learned approximation to $q_\pi$. This algorithm is called an *all-actions method* because its update involves all of the actions, but right now our interest is on the REINFORCE algorithm where update at time $t$ involves only $A_t$, the one action taken at time $t$. 

We introduce $A_t$ by replacing a sum over the random variables possible values with an expectation under $\pi$, and then sampling the expectation. (13.6) involves an appropriate sum over the actions but each term isn't weighted by $\pi(a | s, \theta)$ as is needed for an expectation under $\pi$. So we introduce a weighting without changing the equality by multiplying and then dividing the summed terms by $\pi(a | S_t, \theta)$. Continuing from (13.6):

$\nabla J(\theta) = \mathbb{E}_\pi[\sum_a \pi(a | S_t, \theta) q_\pi(S_t, a) \frac{\nabla \pi(a | S_t, \theta)}{\pi(a |S_t, \theta}$  
$\nabla J(\theta) = \mathbb{E}_\pi[q_\pi(S_t, A_t) \frac{\nabla \pi(A_t | S_t, \theta)}{\pi(A_t |S_t, \theta} \quad $ (replacing a by the sample $A_t ~ \pi$.  
$\nabla J(\theta) = \mathbb{E}_\pi[G_t \frac{\nabla \pi(A_t| S_t, \theta)}{\pi(A_t |S_t, \theta)} \quad \quad $ (because $\mathbb{E}_\pi[G_t | S_t, A_t] = q_\pi(S_t, A_t))$  

$G_t$ is the return. The final expression is exactly what we need, a quantity that can be sampled on each step and it has an expectation equal to the gradient. Using this to set up our generic stochastic gradient ascent algorithm yields the REINFORCE update:

$\theta_{t+1} \stackrel{.}{=} \theta_t + \alpha G_t \frac{\nabla \pi(A_t| S_t, \theta)}{\pi(A_t |S_t, \theta)}$

This update rule is appealing because each update is incremental and proportional to the produce of the return $G_t$ and a vector. The vector is the gradient of the probability of choosing the action actually taken divided by the probability of taking that action. Vector is the direction in parameter space that most increases the probability of picking action $A_t$ in state $S_t$. 

REINFORCE uses the complete return from time $t$, which includes all future rewards up through the end of the episode. In this way, REINFORCE is a Monte Carlo algorithm, and it is only well defined for the episodic case with all updates made after the episode is completed. 

REINFORCE: Monte-Carlo policy gradient control (episodic) for $\pi_*$:  

Input: a differentiable policy parameterization $\pi(a | s, \theta)$  
Parameter: step-size $\alpha > 0$  
Initialize policy parameter: $\theta \in \mathbb{R}^{d'}$. i.e., to 0  

Loop for each episode:  
$\quad$ Generate episode: $S_0, A_0, R_1, \dots, S_{T-1}, A_{T-1}, R_T$  
$\quad$ Loop for each step of the episode: $t = 0, 1, \dots, T-1$:  
$\quad$$\quad$ $G \leftarrow \sum_{k=t+1}^{T} \gamma^{k-t-1} R_k$  
$\quad$$\quad$ $\theta \leftarrow \theta + \alpha \gamma^t G \nabla \space \ln \pi(A_t | S_t, \theta)$

REINFORCE has good convergence guarantees, it is constructed so that the expected update over an episode is in the same direction as the performance gradient. This will ensure a performance improvement for sufficiently small $\alpha$ and convergence to local optimum under standard stochastic approximation conditions for decreasing $\alpha$. As a Monte Carlo methods, REINFORCE may be of high variance and thus learn slowly.

### 13.4 REINFORCE with Baseline

Policy gradient theorem can be generalized to include a comparison of the action value to arbitrary baseline $b(s)$. 

$\nabla J(s) \propto \sum_s \mu(s) \sum_a (q_\pi(s, a) - b(s)) \nabla \pi(a | s, \theta) \quad \quad (13.10)$

Baseline can be any function or even a random variable as long as it doesn't vary with $a$. The equation will stay valid because the subtracted quantity is equal to zero.

$\sum_a b(s) \nabla \pi(a | s, \theta) = b(s) \nabla \sum_a \pi(a | s, \theta) = b(s) \nabla 1 = 0$

Policy gradient rule with basline (13.10) can be used to derive a new update rule that is similar to the one in the previous section. Update rule we get is a new version of the REINFORCE algorithm that includes now a general baseline.

$\theta_{t+1} \stackrel{.}{=} \theta_t + \alpha (G_t - b(S_t)) \frac{\nabla \pi (A_t | S_t, \theta_t)}{\pi (A_t | S_t, \theta_t)} \quad \quad (13.11)$

